# Parcial 2 - Healthcare


In [23]:
import io
import os
import zipfile
from inspect import cleandoc
from pathlib import Path
from pprint import pprint
from typing import Final

import pandas as pd
import requests
try:
  from dotenv import load_dotenv
except ImportError:
  !pip install dotenv
  from dotenv import load_dotenv

from IPython.display import Markdown

try:
  from pymongo.errors import OperationFailure
  from pymongo.mongo_client import MongoClient
  from pymongo.server_api import ServerApi
except ImportError:
  !pip install pymongo
  from pymongo.errors import OperationFailure
  from pymongo.mongo_client import MongoClient
  from pymongo.server_api import ServerApi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 20.5 MB/s eta 0:00:00


In [24]:
load_dotenv()

HOST: Final[str] = os.getenv("MONGO_HOST", "localhost")
PORT: Final[int] = int(os.getenv("MONGO_PORT", "27017"))
USER: Final[str] = os.getenv("MONGO_USER", "root")
PASSWORD: Final[str] = os.getenv("MONGO_PASSWORD", "")
DATABASE: Final[str] = os.getenv("MONGO_DATABASE", "healthcare")

In [25]:
def remove_file_and_folder(file_path: Path) -> None:
    """Elimina un archivo o carpeta de forma recursiva.

    Args:
        file_path (Path): Ruta del archivo o carpeta a eliminar.

    """
    if not file_path.exists():
        return

    if file_path.is_file():
        file_path.unlink()
        return

    if file_path.is_dir():
        for child in file_path.iterdir():
            remove_file_and_folder(child)

        file_path.rmdir()
        return

Para el presente trabajo, se están siguiendo las siguientes
[instrucciones](https://konradlorenzedu-my.sharepoint.com/:w:/g/personal/jorgee_hernandezr_konradlorenz_edu_co/ETKV_mSbI5ZPn8SkyIr86V4BEetglP3c6FnfNJBQwKd6uQ?rtime=1YKDe6GY3Ug).
A su vez, el dataset de los dos presentados, se ha decido trabajar solo con el
[siguiente](https://www.kaggle.com/datasets/aranyogeshm/healthcare-dataset/data).


In [26]:
dataset_url: Final[str] = (
    "https://www.kaggle.com/api/v1/datasets/download/aranyogeshm/healthcare-dataset"
)

In [27]:
response = requests.get(dataset_url, timeout=10)

data_path: Final[Path] = Path("data")

# Unzip the datasets and verify the contents
with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
    zip_file.extractall(data_path)

items: list[Path] = [*data_path.iterdir()]

if len(items) != 2:  # noqa: PLR2004
    raise ValueError("Multiple files downloaded, expected only two")  # noqa: EM101, TRY003

# Read healthcare test and train datasets
raw_csv_paths: tuple[Path, Path] = (
    data_path / "healthcareTest.csv",
    data_path / "healthcareTrain.csv",
)

Se hará un _merge_ de los datasets de `healthcareTrain` y `healthcareTest`. Esto
con el fin de contar más datos para la visualización, todo esto teniendo en
cuenta que no se realizará un modelo.


In [28]:
dfs: list[pd.DataFrame] = []

for raw_csv_path in raw_csv_paths:
    df = pd.read_csv(raw_csv_path)
    dfs.append(df)

# Remove the data folder and its contents
remove_file_and_folder(data_path)

df = pd.concat(dfs, ignore_index=True)
df

,patIndex,pdc,num_ip_post,total_los_post,num_op_post,num_er_post,num_ndc_post,num_gpi6_post,adjust_total_30d_post,generic_rate_post,...,brand_cost,ratio_G_total_cost,numofgen_post,numofbrand_post,generic_cost_post,brand_cost_post,ratio_G_total_cost_post,pdc_80_flag,drug_class,patient_key
0,2,0.333333,0,0,4,0,15,5,14.466667,0.101382,...,2984.927229,0.010155,2,13,196.359216,3001.501507,0.061403,0,*ANTIDIABETICS*,168
1,5,0.866667,0,0,5,0,16,4,18.000000,0.888889,...,0.000000,1.000000,14,2,671.755173,735.661568,0.477297,1,*ANTIDIABETICS*,499
2,21,0.500000,0,0,0,0,8,6,8.000000,0.875000,...,0.000000,1.000000,7,1,50.160767,41.220633,0.548917,0,*ANTIDIABETICS*,1830
3,22,0.977778,0,0,9,0,40,9,42.533333,0.835423,...,1345.104492,0.339094,33,7,842.908516,1695.649323,0.332042,1,*ANTIDIABETICS*,1852
4,33,0.527778,0,0,6,0,28,7,28.000000,0.964286,...,0.000000,1.000000,27,1,1163.290225,6.514435,0.994431,0,*ANTIDIABETICS*,3369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1717,3042,0.166667,1,4,5,0,4,3,3.266667,1.000000,...,0.000000,NaN,4,0,10.571707,0.000000,1.000000,0,*ANTIDIABETICS*,414581
1718,460,0.333333,0,0,0,0,6,3,7.166667,1.000000,...,75.754592,0.811928,6,0,142.484006,0.000000,1.000000,0,*ANTIDIABETICS*,58682
1719,342,0.700000,0,0,12,0,32,16,34.100000,0.648094,...,2278.143238,0.061978,22,10,206.516778,2495.125983,0.076441,0,*ANTIDIABETICS*,42540
1720,1784,0.950000,0,0,6,0,17,8,37.666667,0.796460,...,72.851186,0.942509,14,4,903.283332,486.702794,0.649851,1,*ANTIDIABETICS*,250886


## Análisis exploratorio


El conjunto de datos consolidado consta de 96 variables y 1 722 registros.

Las variables se agrupan en siete categorías según su función y tipo de dato:

1. Características demográficas Variables para segmentar poblaciones según edad,
   género, región y tipo de plan/pago:

- `age_grpN` (o `age_cat`): rango de edad y perfil de riesgo.
- `sexN`: sexo del paciente.
- `regionN`: zona geográfica.
- `idx_prodtypeN`: tipo de plan de salud.
- `idx_paytypN`: pagador primario.

2. Comorbilidades clave Indicadores binarios de presencia/ausencia, priorizando
   las más prevalentes o de mayor coste/impacto:

   - _Metabólicas:_ `DIABETES`, `DYSLIPIDEMIA`, `THYROID`
   - _Cardiovasculares:_ `HYPERTENSION`, `MI_CAD`, `CHF`, `CARDIAC_ARRYTHMIA`,
     `CARDIAC_VALVULAR`, `PERIPHERAL_VASCULAR`
   - _Respiratorias:_ `COPD`, `ASTHMA`
   - _Neurológicas y mentales:_ `DEMENTIA`, `EPILEPSY_SEIZURE`, `DEPRESSION`,
     `SCHIZOPHRENIA`, `SLEEP_DISORDERS`
   - _Renales y hepáticas:_ `CHRONIC_KIDNEY`, `LIVER_GALLBLADDER_PANCREAS`,
     `RENAL_FAILURE`, `HEPATITIS`
   - _Cáncer:_ `Solid_Tumor`, `Leukemia_Lymphoma`, `Other_Cancer`,
     `Cancer_In_Situ`, `Metastatic`.

3. Riesgo y severidad Medidas de comorbilidad y adherencia:

   - `pre_CCI`: Índice de Charlson antes del evento índice.
   - `pdc` y `pdc_cat`: grado de adherencia al tratamiento.
   - `pdc_80_flag`: indicador de adherencia ≥ 80 %.

4. Uso de servicios y costes

   - _Antes del evento índice (baseline):_

     - `num_ip`, `num_er`, `num_op`, `num_ndc`, `num_gpi6`
     - `pre_total_cost`, `pre_medical_cost` y sus transformaciones logarítmicas
       (`log_pre_*`).

   - _Después del evento índice (impacto y alertas):_

     - `num_ip_post`, `num_er_post`, `num_op_post`, `num_ndc_post`,
       `num_gpi6_post`
     - `post_total_cost`, `post_medical_cost` y sus logs (`log_*_post`).

   - _Eficiencia en genéricos:_

     - `generic_rate` y `generic_rate_post`
     - `ratio_G_total_cost` y `ratio_G_total_cost_post`.

5. Costes directos para el paciente

   - `idx_copay` y su transformación logarítmica (`log_idx_copay`).
   - `out_of_pocket` (cuando está disponible, equivalente en algunos casos a
     `idx_copay`).

6. Tratamientos y clases de fármacos Variables para analizar el consumo
   farmacológico:

   - `drug_class`: categoría terapéutica.
   - `numofgen`, `numofbrand`, `generic_cost`, `brand_cost` (antes y después del
     índice).

7. Identificadores

   - `patIndex` y `patient_key`: únicamente para el enlace de registros.


In [29]:
df_types = df.dtypes.to_frame()
df_types.columns = ["**Tipo**"]
df_types = df_types.T
df_types.index.name = "Columna"
Markdown(df_types.to_markdown())

| Columna   | patIndex   | pdc     | num_ip_post   | total_los_post   | num_op_post   | num_er_post   | num_ndc_post   | num_gpi6_post   | adjust_total_30d_post   | generic_rate_post   | post_ip_flag   | post_er_flag   | pdc_cat   | post_ip_cost   | post_er_cost   | post_rx_cost   | post_op_cost   | post_total_cost   | post_medical_cost   | idx_copay   | age_grpN   | sexN   | regionN   | idx_prodtypeN   | idx_paytypN   | age_cat   | log_idx_copay   | ALCOHOL_DRUG   | ASTHMA   | CARDIAC_ARRYTHMIA   | CARDIAC_VALVULAR   | CEREBROVASCULAR   | CHRONIC_KIDNEY   | CHRONIC_PAIN_FIBRO   | CHF   | COPD   | DEMENTIA   | DEPRESSION   | DIABETES   | DYSLIPIDEMIA   | EPILEPSY_SEIZURE   | HEPATITIS   | HIV_AIDS   | HYPERTENSION   | LIVER_GALLBLADDER_PANCREAS   | MI_CAD   | OSTEOARTHRITIS   | PARALYSIS   | PEPTIC_ULCER   | PERIPHERAL_VASCULAR   | RENAL_FAILURE   | RHEUMATOLOGIC   | SCHIZOPHRENIA   | SLEEP_DISORDERS   | SMOKING   | THYROID   | Solid_Tumor   | Metastatic   | Leukemia_Lymphoma   | Other_Cancer   | Cancer_In_Situ   | pre_CCI   | pre_ip_cost   | pre_er_cost   | pre_rx_cost   | pre_op_cost   | pre_total_cost   | pre_medical_cost   | num_ip   | total_los   | num_op   | num_er   | num_ndc   | num_gpi6   | adjust_total_30d   | generic_rate   | pre_ip_flag   | pre_er_flag   | log_pre_ip_cost   | log_pre_er_cost   | log_pre_op_cost   | log_pre_rx_cost   | pre_total_cat   | numofgen   | numofbrand   | generic_cost   | brand_cost   | ratio_G_total_cost   | numofgen_post   | numofbrand_post   | generic_cost_post   | brand_cost_post   | ratio_G_total_cost_post   | pdc_80_flag   | drug_class   | patient_key   |
|:----------|:-----------|:--------|:--------------|:-----------------|:--------------|:--------------|:---------------|:----------------|:------------------------|:--------------------|:---------------|:---------------|:----------|:---------------|:---------------|:---------------|:---------------|:------------------|:--------------------|:------------|:-----------|:-------|:----------|:----------------|:--------------|:----------|:----------------|:---------------|:---------|:--------------------|:-------------------|:------------------|:-----------------|:---------------------|:------|:-------|:-----------|:-------------|:-----------|:---------------|:-------------------|:------------|:-----------|:---------------|:-----------------------------|:---------|:-----------------|:------------|:---------------|:----------------------|:----------------|:----------------|:----------------|:------------------|:----------|:----------|:--------------|:-------------|:--------------------|:---------------|:-----------------|:----------|:--------------|:--------------|:--------------|:--------------|:-----------------|:-------------------|:---------|:------------|:---------|:---------|:----------|:-----------|:-------------------|:---------------|:--------------|:--------------|:------------------|:------------------|:------------------|:------------------|:----------------|:-----------|:-------------|:---------------|:-------------|:---------------------|:----------------|:------------------|:--------------------|:------------------|:--------------------------|:--------------|:-------------|:--------------|
| **Tipo**  | int64      | float64 | int64         | int64            | int64         | int64         | int64          | int64           | float64                 | float64             | int64          | int64          | int64     | float64        | float64        | float64        | float64        | float64           | float64             | float64     | int64      | int64  | int64     | int64           | int64         | int64     | float64         | int64          | int64    | int64               | int64              | int64             | int64            | int64                | int64 | int64  | int64      | int64        | int64      | int64          | int64              | int64       | int64      | int64          | int64                        | int64    | int64            | int64       | int64          | int64                 | int64           | int64           | int64           | int64             | int64     | int64     | int64         | int64        | int64               | int64          | int64            | int64     | float64       | float64       | float64       | float64       | float64          | float64            | int64    | int64       | int64    | int64    | int64     | int64      | float64            | float64        | int64         | int64         | float64           | float64           | float64           | float64           | int64           | int64      | int64        | float64        | float64      | float64              | int64           | int64             | float64             | float64           | float64                   | int64         | object       | int64         |

## Procesamiento de datos


Verificamos si existen datos nulos o faltantes en el dataset.


In [30]:
missing_values = df.isna().sum()[df.isna().sum() > 0]
missing_values_percentage = (missing_values / df.shape[0] * 100).sort_values(
    ascending=False
)

prev_rows_number = df.shape[0]
df = df.dropna()


Markdown(
    cleandoc(
        """
La cantidad de valores nulos en relación a la cantidad de filas es:

- `ratio_G_total_cost`: $72$
- `ratio_G_total_cost_post`: $28$

Lo que representa un porcentaje de:

- `ratio_G_total_cost`: $4.18\%$
- `ratio_G_total_cost_post`: $1.63\%$

Dado estos bajos porcentajes de valores nulos, se procederá a realizar la amputación de los mismos.

La cantidad de filas originales es: $1722$.
La cantidad de filas luego de eliminar los valores nulos es: $1628$
"""
    )
)

La cantidad de valores nulos en relación a la cantidad de filas es:

- `ratio_G_total_cost`: $72$
- `ratio_G_total_cost_post`: $28$

Lo que representa un porcentaje de:

- `ratio_G_total_cost`: $4.18\%$
- `ratio_G_total_cost_post`: $1.63\%$

Dado estos bajos porcentajes de valores nulos, se procederá a realizar la amputación de los mismos.

La cantidad de filas originales es: $1722$.
La cantidad de filas luego de eliminar los valores nulos es: $1628$

Ahora, verificamos si existen datos duplicados en el dataset.


In [31]:
Markdown(f"No existe ningún valor duplicado: ${df.duplicated().sum()}$")

No existe ningún valor duplicado: $0$

Eliminamos columnas innecesarias para el análisis, como `patIndex`,
`patient_key` y `drug_class` (este último solo cuenta con un valor único por lo
que no aporta al análisis).


In [32]:
df = df.drop(["patIndex", "patient_key", "drug_class"], axis=1)

Teniendo así finalmente la siguiente cantidad de columnas y filas.


In [33]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 1628 entries, 0 to 1721
Data columns (total 93 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   pdc                         1628 non-null   float64
 1   num_ip_post                 1628 non-null   int64  
 2   total_los_post              1628 non-null   int64  
 3   num_op_post                 1628 non-null   int64  
 4   num_er_post                 1628 non-null   int64  
 5   num_ndc_post                1628 non-null   int64  
 6   num_gpi6_post               1628 non-null   int64  
 7   adjust_total_30d_post       1628 non-null   float64
 8   generic_rate_post           1628 non-null   float64
 9   post_ip_flag                1628 non-null   int64  
 10  post_er_flag                1628 non-null   int64  
 11  pdc_cat                     1628 non-null   int64  
 12  post_ip_cost                1628 non-null   float64
 13  post_er_cost                1628 non-n

,pdc,num_ip_post,total_los_post,num_op_post,num_er_post,num_ndc_post,num_gpi6_post,adjust_total_30d_post,generic_rate_post,post_ip_flag,...,numofbrand,generic_cost,brand_cost,ratio_G_total_cost,numofgen_post,numofbrand_post,generic_cost_post,brand_cost_post,ratio_G_total_cost_post,pdc_80_flag
count,1628.000000,1628.000000,1628.000000,1628.00000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,...,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000,1628.000000
mean,0.644711,0.090909,0.466830,7.37715,0.211302,17.824324,7.138821,23.346704,0.755283,0.074939,...,3.248771,263.019948,1012.761828,0.561949,13.012899,4.866708,323.305949,1437.473057,0.467706,0.399877
std,0.282358,0.345793,2.636933,8.86061,0.626849,12.648910,4.074794,17.314394,0.240959,0.263373,...,4.663755,437.551710,2726.323850,0.405131,9.856647,5.668581,505.565571,3169.288921,0.393637,0.490023
min,0.022222,0.000000,0.000000,0.00000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,1.408040,0.000000,0.000252,1.000000,0.000000,0.040510,0.000000,0.000488,0.000000
25%,0.450000,0.000000,0.000000,3.00000,0.000000,9.000000,4.000000,10.300000,0.595734,0.000000,...,0.000000,39.524980,0.000000,0.145901,6.000000,0.000000,69.651621,0.000000,0.091003,0.000000
50%,0.672222,0.000000,0.000000,5.00000,0.000000,15.000000,6.000000,19.433333,0.800396,0.000000,...,1.000000,116.376099,135.157849,0.531003,11.000000,3.000000,150.515371,411.978917,0.324676,0.000000
75%,0.911111,0.000000,0.000000,9.00000,0.000000,23.000000,9.000000,31.666667,1.000000,0.000000,...,5.000000,302.033142,1135.682722,1.000000,17.000000,7.250000,352.575436,1870.305814,1.000000,1.000000
max,1.000000,3.000000,61.000000,180.00000,6.000000,88.000000,26.000000,111.600000,1.000000,1.000000,...,36.000000,5775.928188,50024.814630,1.000000,74.000000,42.000000,5855.110311,66315.417270,1.000000,1.000000


## Análisis univariado

Cómo es sabido, el dataset cuenta con más de 90 columnas, si bien, bastantes nos pueden aportar información, tambien hay que detectar algunas que no, esto, para evitar trabajo sobre analizar variables de más...

## Proceso ETL


Como parte del proceso ETL, ya se ha realizado lo siguiente:

- Extracción:  
  Se ha extraído el dataset de Kaggle y se ha guardado en un archivos CSV que
  posteriormente son cargados en un DataFrame de Pandas.

- Transformación:  
  En este caso, se ha eliminado las filas con datos nulos o faltantes, así como
  las columnas que no aportaban información relevante para el análisis.

Como último paso pendiente, se debe cargar el DataFrame en una base de datos de
MongoDB. A partir de ello, se podrá conectar Power BI a la base de datos y
realizar el análisis de datos correspondiente.


### Carga


Primero procederemos a realizar la conexión a la base de datos de MongoDB.


In [ ]:
uri = f"mongodb+srv://{USER}:{PASSWORD}@{HOST}/?retryWrites=true&w=majority&appName=healthCare"
client = MongoClient(uri, server_api=ServerApi("1"))

try:
    client.admin.command("ping")
    print("Pinged your deployment. You successfully connected to MongoDB!")
except OperationFailure as e:
    raise e from e

db = client[DATABASE]
collection = db["healthcare"]

Realizamos la carga del DataFrame a la base de datos de MongoDB, en la colección
`healthcare`.


In [ ]:
collection.drop()
final_df = df.reset_index(drop=True)

collection.insert_many(final_df.to_dict(orient="records"))
None

Mostramos los primeros dos registros de la colección `healthcare` para verificar
que se ha realizado la carga correctamente.


In [ ]:
two_records = collection.find().limit(2)

pprint(list(two_records))  # noqa: T203